In [1]:
from collections import Counter
import re
from __future__ import division
import numpy as np
import string

In [2]:
# pos_tags = [u'CC', u'CD', u'DT', u'EX', u'FW', u'IN', u'JJ', u'JJR', u'JJS', u'LS', u'MD',
#             u'NN', u'NP', u'NPS', u'NNS', u'PDT', u'POS', u'PP', u'PP$', u'RB', u'RBR',
#             u'RBS', u'RP', u'SYM', u'TO', u'UH', u'VB', u'VBD', u'VBG', u'VBN', u'VBP', u'VBZ',
#             u'WDT', u'WP', u'WP$', u'WRB']
with open("POS.train", 'r') as train: 
    text = train.read()
# with open("POS.train", 'r') as train: 
#     lines = train.readlines()
# #     pos_tags_single = re.findall('(/[A-Z]{2,3}\$*)', text)
# #     pos_tags = re.findall('(/[A-Z]{2,3}\$*\|*[A-Z]{2,3}\$*)', text)
# # pos_tags = set(pos_tags).union(set(pos_tags_single))

# # pos_tags = [x[1:] for x in pos_tags]
# print(pos_tags,len(pos_tags))

In [3]:
# # c(wi,ti)
# c_wi_ti = Counter() # in the order wi/ti
# c_ti = Counter()
# c_ti_tim1 = Counter() # in order t_i/t_{i-1}
# num_lines = 0
# with open("POS.train", 'r') as train:
#     for line in train:
#         num_lines += 1
#         c_wi_ti.update(Counter(line[:-2].split(' ')))
#         # currently, this gets all things after / even if it's word1\/word2 which i dont want (such as Cities\/ABC/NP)
#         pos_order = re.findall('(/[A-Z]{2,3}\$*)', line) # finds things like /DT
#         c_ti.update(Counter(pos_order))
#         for ind, pos in enumerate(pos_order):
#             if ind == 0:
#                 # pos[1:] bc it starts w/ "/"
#                 c_ti_tim1.update(Counter([pos[1:] + '/<s>']))
#             else:
#                 c_ti_tim1.update(Counter([pos[1:] + pos_order[ind-1]]))
# c_ti['/<s>'] = num_lines
# del c_ti['/ABC'] # GET RID OF this mistake until I fix my regex

In [4]:
# c_ti

In [5]:
# # this does not include punctuation... should I? It seems like a hard rule lol
# import time
# start = time.time()
# p_wi_ti = {}
# # for POS, POS_count in c_ti.items():
# #     for key, val in c_wi_ti.items():
# #         if POS in key:
# #             p_wi_ti[key] = val / POS_count
# for key, val in c_wi_ti.items():
#     pos = '/' + key.split('/')[1]
#     try:
#         p_wi_ti[key] = val / c_ti[pos]
#     except: # need to fix this but for now make it .001
#         p_wi_ti[key] = .001
# # print(len(p_wi_ti), p_wi_ti['the/DT'], time.time()-start)


In [6]:
# p_ti_tim1 = {}
# # <s>
# for key, val in c_ti_tim1.items():
#     pos = '/' + key.split('/')[-1] # we want total count of this pos for denominator
#     try:
#         p_ti_tim1[key] = val / c_ti[pos]
#     except: # need to fix this but for now make it .001
#         p_ti_tim1[key] = .001



In [7]:
from collections import Counter

# Gather counts before implementing the algorithm
# bigram_count 
unigram = []
bigram = []
bigram_count= {}
i = 0

with open ('POS.train','r') as training:
	lines = training.readlines()
	start = len(lines)

	for line in lines:
		line = line.split()
		line.append('start')
		for item in line:
			unigram.append(item.split('/')[-1])

	for word in unigram:
		if i < len(unigram)-1:
			bigram.append(unigram[i] + ' '+unigram[i+1])
			i+=1

	new_bigram = [x for x in bigram if not x.endswith("start")]

	for item in new_bigram:
		if item not in bigram_count:
			bigram_count[item]=1
		else:
			bigram_count[item]+=1

# tag_frequency, pos_word_prob 
pos = []
with open ("POS.train",'r') as training:
	string = training.read()
	
	splited_string = string.split()

	for item in splited_string:
		pos.append(item.split('/')[-1]) #Choose the last one when there are multiple slashes.

	tag_frequency = Counter(pos)
	tag_frequency.update({'start':start})

	pos_word_count = Counter(splited_string)
	pos_word_prob = pos_word_count

	for item in pos_word_count:
		pos_word_prob[item] = pos_word_count[item]/float(tag_frequency[item.split('/')[-1]])

# bigram_prob
bigram_prob = {}

for key,value in bigram_count.items():
	bigram_prob[key]=value/float(tag_frequency[key.split()[0]])

In [8]:
all_tags = []
for key in bigram_prob.keys(): # in order i-1 i
    all_tags.extend(key.split())

uniq_tags = set(all_tags)

In [9]:
pos_word_prob

Counter({'dreamy/JJ': 6.292870178088226e-05,
         'Isabelle/NP': 4.1967433271781096e-05,
         'unsupported/JJ': 0.00012585740356176452,
         'stir/VB': 0.00014734050390452334,
         'yellow/JJ': 6.292870178088226e-05,
         'clearance/NN': 0.0002068313438127881,
         'Cathryn/NP': 8.393486654356219e-05,
         'rarefied/VBN': 0.00019227071716977504,
         'Bordeaux/NP': 8.393486654356219e-05,
         'novel/JJ': 6.292870178088226e-05,
         'Eubank/NP': 4.1967433271781096e-05,
         'but/RB': 0.0001281722635221738,
         'fledgling/NN': 2.9547334830398298e-05,
         '250,000-square-foot/JJ': 6.292870178088226e-05,
         'examiners/NNS': 0.00019401151134967342,
         'seeks/VBZ': 0.0018601190476190475,
         'certificate/NN': 0.00011818933932159319,
         'Donoghue/NP': 4.1967433271781096e-05,
         'Patterson/NP': 4.1967433271781096e-05,
         'worries/NNS': 0.00012934100756644895,
         'Next/JJ': 6.292870178088226e-05,
    

In [10]:
# # init
# for word in text.split():
#     val = word.split('/')[0]
# #     print(word, val)
#     init_word_vec = []
#     for tag in pos_tags:
#         wi_ti = val + '/' + tag
#         print(wi_ti)
#         ti_tim1 = tag + '/<s>'
#         try:
# #             print(p_wi_ti[wi_ti])
#             init_word_vec.append(p_wi_ti[wi_ti]*p_ti_tim1[ti_tim1])
#             # if wi_ti
#         except:
#             init_word_vec.append(0)
#     break

In [11]:
# total_vec = [init_word_vec]
# back_vec=[0]
# print(total_vec)

In [12]:
# import numpy as np

# for word_ind, word in enumerate(text.split()[1:100]): # for each word
#     val = word.split('/')[0] 
#     word_vec = []
#     for pos_ind, tag in enumerate(pos_tags): # for each tag
#         # get key for p(wi|ti)
#         wi_ti = val + '/' + tag
#         tag_probs = []
#         for cond_tag in pos_tags: 
#             # get key for p(ti|t_j) where j is over all tags
#             ti_tim1 = tag + '/' + cond_tag
#             try:
#                 tag_probs.append(p_ti_tim1[ti_tim1])
#             except:
#                 tag_probs.append(0)
        
#         # previous score vector * vector of conditional tag probs
#         vec_to_max = np.array(total_vec[word_ind]) * np.array(tag_probs)
#         try:
#             word_vec.append(p_wi_ti[wi_ti] * max(vec_to_max))
#         except:
#             word_vec.append(0)
#     back_vec.append(np.argmax(vec_to_max))
#     total_vec.append(word_vec)



In [13]:
# print(total_vec)

In [14]:
# sentence[0:17]

In [15]:
trans_p = np.zeros(shape=(len(uniq_tags), len(uniq_tags)))
print(trans_p.shape, len(trans_p))

((56, 56), 56)


In [16]:
for col_ind, tag in enumerate(uniq_tags):
    for row_ind, cond_tag in enumerate(uniq_tags):
#         if row_ind == 0:
#             ti_tim1 = tag + '/<s>'
#         else:
#         ti_tim1 = tag + '/' + cond_tag
        ti_tim1 = tag + ' ' + cond_tag
        try:
#             print('trans_p at {},{} is {} w/ val {}'.format(row_ind, col_ind,
#                                                             ti_tim1, p_ti_tim1[ti_tim1]))
#             trans_p[row_ind, col_ind] = p_ti_tim1[ti_tim1]
            print('trans_p at {},{} is {} w/ val {}'.format(row_ind, col_ind,
                                                            ti_tim1, bigram_prob[ti_tim1]))
            trans_p[row_ind, col_ind] = bigram_prob[ti_tim1]
        except:
            print('trans_p at {},{} is {} w/ val {}'.format(row_ind, col_ind,
                                                            ti_tim1, 0))
        

trans_p at 0,0 is VBD|RB VBD|RB w/ val 0
trans_p at 1,0 is VBD|RB VBG w/ val 0
trans_p at 2,0 is VBD|RB VBD w/ val 0
trans_p at 3,0 is VBD|RB VBG|NN w/ val 0
trans_p at 4,0 is VBD|RB VB w/ val 1.0
trans_p at 5,0 is VBD|RB POS w/ val 0
trans_p at 6,0 is VBD|RB '' w/ val 0
trans_p at 7,0 is VBD|RB VBP w/ val 0
trans_p at 8,0 is VBD|RB WDT w/ val 0
trans_p at 9,0 is VBD|RB JJ w/ val 0
trans_p at 10,0 is VBD|RB WP w/ val 0
trans_p at 11,0 is VBD|RB VBZ w/ val 0
trans_p at 12,0 is VBD|RB DT w/ val 0
trans_p at 13,0 is VBD|RB NNS|NN w/ val 0
trans_p at 14,0 is VBD|RB PP w/ val 0
trans_p at 15,0 is VBD|RB RBR|JJR w/ val 0
trans_p at 16,0 is VBD|RB RP w/ val 0
trans_p at 17,0 is VBD|RB $ w/ val 0
trans_p at 18,0 is VBD|RB NN w/ val 0
trans_p at 19,0 is VBD|RB start w/ val 0
trans_p at 20,0 is VBD|RB ) w/ val 0
trans_p at 21,0 is VBD|RB ( w/ val 0
trans_p at 22,0 is VBD|RB FW w/ val 0
trans_p at 23,0 is VBD|RB , w/ val 0
trans_p at 24,0 is VBD|RB . w/ val 0
trans_p at 25,0 is VBD|RB RB|IN w/ va

trans_p at 7,18 is NN VBP w/ val 0.00410707954143
trans_p at 8,18 is NN WDT w/ val 0.00889374778395
trans_p at 9,18 is NN JJ w/ val 0.00850963243115
trans_p at 10,18 is NN WP w/ val 0.00260016546508
trans_p at 11,18 is NN VBZ w/ val 0.0443800969153
trans_p at 12,18 is NN DT w/ val 0.00691407635031
trans_p at 13,18 is NN NNS|NN w/ val 0
trans_p at 14,18 is NN PP w/ val 0.00419572154592
trans_p at 15,18 is NN RBR|JJR w/ val 0
trans_p at 16,18 is NN RP w/ val 0.000561399361778
trans_p at 17,18 is NN $ w/ val 0.000206831343813
trans_p at 18,18 is NN NN w/ val 0.12625576173
trans_p at 19,18 is NN start w/ val 0
trans_p at 20,18 is NN ) w/ val 0.00192057676398
trans_p at 21,18 is NN ( w/ val 0.00159555608084
trans_p at 22,18 is NN FW w/ val 8.86420044912e-05
trans_p at 23,18 is NN , w/ val 0.111895757003
trans_p at 24,18 is NN . w/ val 0.106872710082
trans_p at 25,18 is NN RB|IN w/ val 0
trans_p at 26,18 is NN TO w/ val 0.0381751566009
trans_p at 27,18 is NN LS w/ val 0
trans_p at 28,18 is N

In [17]:
# obs_matrix = np.zeros(shape=(len(pos_tags), len(text.split())))
obs_matrix = np.zeros(shape=(len(uniq_tags), len(text.split())))


print(obs_matrix.shape)

(56, 242261)


In [18]:
# # maybe do this for each sentence?? Also need to update p_wi_ti and p_ti|tim1 for punctuation
# for line in lines[0:1]:
# #     print(line)
#     sentence = []
#     # need to do this 
#     for col_ind, txt in enumerate(line.split()):
#         word = txt.split('/')[0]


#         sentence.append(word)
#         for row_ind, pos in enumerate(pos_tags):
#             wi_ti = word + '/' + pos
#             try:
#                 print('obs_mat at {},{} is {} w/ val {}'.format(row_ind, col_ind,
#                                                                 wi_ti, p_wi_ti[wi_ti]))
#                 obs_matrix[row_ind, col_ind] = p_wi_ti[wi_ti]
#             except:
#                 print('obs_mat at {},{} is {} w/ val {}'.format(row_ind, col_ind,
#                                                                 wi_ti, 0))       
#                 pass



In [40]:
# sentence = []
# for col_ind, txt in enumerate(text.split()[0:17]):
#     word = txt.split('/')[0]
# #     if word.strip(string.punctuation) == '':
# #         continue

#     sentence.append(word)
# #     for row_ind, pos in enumerate(pos_tags):
#     for row_ind, pos in enumerate(uniq_tags):
#         wi_ti = word + '/' + pos
#         try:
# #             print('obs_mat at {},{} is {} w/ val {}'.format(row_ind, col_ind,
# #                                                             wi_ti, p_wi_ti[wi_ti]))
# #             obs_matrix[row_ind, col_ind] = p_wi_ti[wi_ti]
# #             print('obs_mat at {},{} is {} w/ val {}'.format(row_ind, col_ind,
# #                                                             wi_ti, pos_word_prob[wi_ti]))
#             obs_matrix[row_ind, col_ind] = pos_word_prob[wi_ti]
#         except:
# #             print('obs_mat at {},{} is {} w/ val {}'.format(row_ind, col_ind,
# #                                                             wi_ti, 0))       
#             pass
word_map.keys()

['Pierre',
 'Vinken',
 ',',
 '61',
 'years',
 'old',
 'will',
 'join',
 'the',
 'board',
 'as',
 'a',
 'nonexecutive',
 'director',
 'Nov.',
 '29',
 '.',
 'Mr.',
 'is',
 'chairman',
 'of',
 'Elsevier',
 'N.V.',
 'Dutch',
 'publishing',
 'group',
 'Rudolph',
 'Agnew',
 '55',
 'and',
 'former',
 'Consolidated',
 'Gold',
 'Fields',
 'PLC',
 'was',
 'named',
 'this',
 'British',
 'industrial',
 'conglomerate',
 'A',
 'form',
 'asbestos',
 'once',
 'used',
 'to',
 'make',
 'Kent',
 'cigarette',
 'filters',
 'has',
 'caused',
 'high',
 'percentage',
 'cancer',
 'deaths',
 'among',
 'workers',
 'exposed',
 'it',
 'more',
 'than',
 '30',
 'ago',
 'researchers',
 'reported',
 'The',
 'fiber',
 'crocidolite',
 'unusually',
 'resilient',
 'enters',
 'lungs',
 'with',
 'even',
 'brief',
 'exposures',
 'causing',
 'symptoms',
 'that',
 'show',
 'up',
 'decades',
 'later',
 'said',
 'Lorillard',
 'Inc.',
 'unit',
 'New',
 'York-based',
 'Loews',
 'Corp.',
 'makes',
 'cigarettes',
 'stopped',
 'using

In [43]:
o_mat = np.zeros(shape=(len(uniq_tags), len(word_map.keys())))
# create obs via uniq words
for col_ind, word in enumerate(word_map.keys()):
#     if word.strip(string.punctuation) == '':
#         continue

#     sentence.append(word)
#     for row_ind, pos in enumerate(pos_tags):
    for row_ind, pos in enumerate(uniq_tags):
        wi_ti = word + '/' + pos
        try:
#             print('obs_mat at {},{} is {} w/ val {}'.format(row_ind, col_ind,
#                                                             wi_ti, p_wi_ti[wi_ti]))
#             obs_matrix[row_ind, col_ind] = p_wi_ti[wi_ti]
#             print('obs_mat at {},{} is {} w/ val {}'.format(row_ind, col_ind,
#                                                             wi_ti, pos_word_prob[wi_ti]))
            o_mat[row_ind, col_ind] = pos_word_prob[wi_ti]
        except:
#             print('obs_mat at {},{} is {} w/ val {}'.format(row_ind, col_ind,
#                                                             wi_ti, 0))       
            pass

In [20]:
# init_state = np.zeros(shape=(len(pos_tags),1))
init_state = np.zeros(shape=(len(uniq_tags), 1))
# print(init_state.shape, sentence)
# obs_matrix[:,0:17]

In [21]:
print(obs_matrix[:,0])

[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   8.39348665e-05   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]


In [22]:
# for ind, tag in enumerate(pos_tags):
for ind, tag in enumerate(uniq_tags):
    try:
#         print('init_state at {}, is {} w/ val {}'.format(ind, tag + '/<s>',
#                                                          p_ti_tim1[tag + '/<s>']))       
#         init_state[ind] = p_ti_tim1[tag + '/<s>']
        print('init_state at {}, is {} w/ val {}'.format(ind, 'start' + " "+ tag,
                                                         bigram_prob['start' + ' ' + tag]))       
        init_state[ind] = bigram_prob['start' + ' ' + tag]
    except:
        pass

init_state at 1, is start VBG w/ val 0.0097
init_state at 2, is start VBD w/ val 0.0007
init_state at 4, is start VB w/ val 0.0034
init_state at 6, is start '' w/ val 0.0584
init_state at 7, is start VBP w/ val 0.0003
init_state at 8, is start WDT w/ val 0.0007
init_state at 9, is start JJ w/ val 0.0365
init_state at 10, is start WP w/ val 0.0022
init_state at 11, is start VBZ w/ val 0.0015
init_state at 12, is start DT w/ val 0.2183
init_state at 14, is start PP w/ val 0.0596
init_state at 17, is start $ w/ val 0.0002
init_state at 18, is start NN w/ val 0.036
init_state at 20, is start ) w/ val 0.0061
init_state at 21, is start ( w/ val 0.0064
init_state at 22, is start FW w/ val 0.0002
init_state at 24, is start . w/ val 0.0075
init_state at 26, is start TO w/ val 0.0032
init_state at 27, is start LS w/ val 0.0014
init_state at 28, is start RB w/ val 0.048
init_state at 29, is start NP w/ val 0.1824
init_state at 30, is start : w/ val 0.001
init_state at 31, is start NNS w/ val 0.03

In [23]:
class Decoder(object):
    def __init__(self, initialProb, transProb, obsProb):
        self.N = initialProb.shape[0]
        self.initialProb = initialProb
        self.transProb = transProb
        self.obsProb = obsProb
        assert self.initialProb.shape == (self.N, 1)
        assert self.transProb.shape == (self.N, self.N)
        assert self.obsProb.shape[0] == self.N

    def Obs(self, obs):
        return self.obsProb[:, obs, None]

    def Decode(self, obs):
        trellis = np.zeros((self.N, len(obs)))
        backpt = np.ones((self.N, len(obs)), 'int32') * -1

        # initialization
        # first row of trans matrix time first column of 
        # emission matrix
        trellis[:, 0] = np.squeeze(self.initialProb * self.Obs(obs[0]))

        for t in xrange(1, len(obs)):
            # trellis[:, t-1, None] is previous column of figure 10.9
            # self.Obs(obs[t]) is the t_th column of our p(w_i|t_i) matrix
            trellis[:, t] = (trellis[:, t-1, None].dot(self.Obs(obs[t]).T) * self.transProb).max(0)
            backpt[:, t] = (np.tile(trellis[:, t-1, None], [1, self.N]) * self.transProb).argmax(0)
        # termination
        tokens = [trellis[:, -1].argmax()]
        for i in xrange(len(obs)-1, 0, -1):
            tokens.append(backpt[tokens[-1], i])
        return tokens[::-1]

In [38]:
decoder = Decoder(initialProb=init_state, transProb=trans_p, obsProb=obs_matrix)
sentence = [x.split('/')[0] for x in lines[0].split()]
states = decoder.Decode(np.arange(state))
# result = np.array(pos_tags)[states].tolist()
result = np.array(list(uniq_tags))[states].tolist()
resultTagged = zip(sentence,result)
print(lines[0])
print(resultTagged)
print(np.arange(len(sentence)))

Pierre/NP Vinken/NP ,/, 61/CD years/NNS old/JJ ,/, will/MD join/VB the/DT board/NN as/IN a/DT nonexecutive/JJ director/NN Nov./NP 29/CD ./. 

[('Pierre', 'NP'), ('Vinken', 'NP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'NN'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'VB'), ('nonexecutive', 'VBD|RB'), ('director', 'VBD|RB'), ('Nov.', 'VBD|RB'), ('29', 'VBD|RB'), ('.', 'VBD|RB')]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]


In [25]:
# sentence = [x.split('/')[0] for x in lines[1].split()]
# states = decoder.Decode(np.arange(18,50))
# # result = np.array(pos_tags)[states].tolist()
# result = np.array(list(uniq_tags))[states].tolist()
# resultTagged = zip(sentence,result)
# print(lines[1])
# print(resultTagged)
# print(np.arange(18, 31))

In [26]:
# '.'.strip(string.punctuation)

# To do/questions
- fix p_wi_ti and p_ti_tim1 to include punctuation
- create obs matrix and run decoder for each sentence,
    - should obs matrix only contain unique words, ie 'janet loves janet' would only have 2 columns?
    - for each sentence, get error??
- testing? p_wi_ti, p_ti_tim1, obs matrix and transition matrix should all be fixed from using the training data? This means I have to create obs matrix for the whole text in the training data?? So basically, in the training file for sentence one, I map each word to a state and in the text file, I find the state of a given word and use that? if a word is new??

In [27]:
# with open("POS.train", 'r') as train:
#     lines = train.readlines()

In [28]:
# create mapping from each word to a state
# with open("POS.train", 'r') as train:
#     for line in train:
words = []
for line in lines:
    for pair in line.split():
        words.append(pair.split('/')[0])


In [29]:
# create ordered word mapping to observation matrix (mainly for testin)
from collections import OrderedDict

word_map = OrderedDict()

ind = 0
for word in words:
    if word not in word_map:
        word_map[word] = ind
        ind += 1



In [45]:
counter = 0
# gets stuck at VBD|RB... maybe when I get a POS1|POS2 just choose the first one
decoder = Decoder(initialProb=init_state, transProb=trans_p, obsProb=o_mat)
for line in lines[0:2]:
    print(line)
    sentence = []
    state = []
    # need to do this 
    for col_ind, txt in enumerate(line.split()):
        word = txt.split('/')[0]
        print('word:state is {}:{}'.format(word, word_map[word]))
        state.append(word_map[word])
        sentence.append(word)
    print(np.array(state))
    states = decoder.Decode(np.array(state))
    result = np.array(list(uniq_tags))[states].tolist()


#     result = np.array(pos_tags)[states].tolist()
    resultTagged = zip(sentence,result)
    print(resultTagged)
    print(lines[counter])
    counter += 1

Pierre/NP Vinken/NP ,/, 61/CD years/NNS old/JJ ,/, will/MD join/VB the/DT board/NN as/IN a/DT nonexecutive/JJ director/NN Nov./NP 29/CD ./. 

word:state is Pierre:0
word:state is Vinken:1
word:state is ,:2
word:state is 61:3
word:state is years:4
word:state is old:5
word:state is ,:2
word:state is will:6
word:state is join:7
word:state is the:8
word:state is board:9
word:state is as:10
word:state is a:11
word:state is nonexecutive:12
word:state is director:13
word:state is Nov.:14
word:state is 29:15
word:state is .:16
[ 0  1  2  3  4  5  2  6  7  8  9 10 11 12 13 14 15 16]
[('Pierre', 'NP'), ('Vinken', 'NP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'NN'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'VB'), ('nonexecutive', 'VBD|RB'), ('director', 'VBD|RB'), ('Nov.', 'VBD|RB'), ('29', 'VBD|RB'), ('.', 'VBD|RB')]
Pierre/NP Vinken/NP ,/, 61/CD years/NNS old/JJ ,/, will/MD join/VB the/DT board/NN as/IN a/DT nonexecutive/JJ dir

OrderedDict([('Pierre', 0),
             ('Vinken', 1),
             (',', 2),
             ('61', 3),
             ('years', 4),
             ('old', 5),
             ('will', 6),
             ('join', 7),
             ('the', 8),
             ('board', 9),
             ('as', 10),
             ('a', 11),
             ('nonexecutive', 12),
             ('director', 13),
             ('Nov.', 14),
             ('29', 15),
             ('.', 16),
             ('Mr.', 17),
             ('is', 18),
             ('chairman', 19),
             ('of', 20),
             ('Elsevier', 21),
             ('N.V.', 22),
             ('Dutch', 23),
             ('publishing', 24),
             ('group', 25),
             ('Rudolph', 26),
             ('Agnew', 27),
             ('55', 28),
             ('and', 29),
             ('former', 30),
             ('Consolidated', 31),
             ('Gold', 32),
             ('Fields', 33),
             ('PLC', 34),
             ('was', 35),
             ('

In [31]:
# # c(wi,ti)
# c_wi_ti = Counter() # in the order wi/ti
# c_ti = Counter()
# c_ti_tim1 = Counter() # in order t_i/t_{i-1}
# num_lines = 0
# with open("POS.train", 'r') as train:
#     for line in train:
#         num_lines += 1
#         c_wi_ti.update(Counter(line[:-2].split(' ')))
#         pos_tags_single = re.findall('(/[A-Z]{2,3}\$*)', line)
#         pos_tags = re.findall('(/[A-Z]{2,3}\$*\|*[A-Z]{2,3}\$*)', line)
#         pos_tags = set(pos_tags).union(set(pos_tags_single))
#         pos_order = [x[1:] for x in pos_tags]        
#         c_ti.update(Counter(pos_order))
#         for ind, pos in enumerate(pos_order):
#             if ind == 0:
#                 c_ti_tim1.update(Counter([pos + '/<s>']))
#             else:
#                 c_ti_tim1.update(Counter([pos + '/' + pos_order[ind-1]]))
# c_ti['<s>'] = num_lines